In [26]:
# Import Library
import os
import shutil
import random
import numpy as np
from PIL import Image
from tqdm import tqdm
import math
import time
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
# Parameter Global & Seed
# Path dataset asli
ORIGINAL_DATASET = "D:/KULIAH/SEMESTER 7/Skripsi/Dataset/dataset_trashnet"

# Dataset hasil resize
RESIZED_DATASET = "D:/KULIAH/SEMESTER 7/Skripsi/Dataset/dataset_resize"

# Dataset final (split + augmentasi)
FINAL_DATASET = "D:/KULIAH/SEMESTER 7/Skripsi/Dataset/Dataset_TrashNet_Final"

# Kelas
CLASSES = ["cardboard", "glass", "metal", "paper", "plastic", "trash"]

# Split ratio
TRAIN_RATIO = 0.7
VAL_RATIO = 0.15
TEST_RATIO = 0.15

# Image size MobileNet
IMG_SIZE = 224

# Reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [3]:
# Fungsi Resize + Padding (Aspect Ratio Preserved)
def resize_with_padding(img, target_size=224):
    w, h = img.size
    scale = target_size / max(w, h)
    new_w = int(w * scale)
    new_h = int(h * scale)

    img = img.resize((new_w, new_h), Image.BILINEAR)

    new_img = Image.new("RGB", (target_size, target_size), (0, 0, 0))
    paste_x = (target_size - new_w) // 2
    paste_y = (target_size - new_h) // 2
    new_img.paste(img, (paste_x, paste_y))

    return new_img

In [4]:
# Resize Seluruh Dataset & Simpan ke Lokal
def resize_dataset():
    for cls in CLASSES:
        src_dir = os.path.join(ORIGINAL_DATASET, cls)
        dst_dir = os.path.join(RESIZED_DATASET, cls)
        os.makedirs(dst_dir, exist_ok=True)

        for img_name in tqdm(os.listdir(src_dir), desc=f"Resizing {cls}"):
            img_path = os.path.join(src_dir, img_name)
            img = Image.open(img_path).convert("RGB")

            resized_img = resize_with_padding(img, IMG_SIZE)
            resized_img.save(os.path.join(dst_dir, img_name))

resize_dataset()

Resizing trash: 100%|████████████████████████████████████████████████████████████████| 137/137 [00:01<00:00, 92.22it/s]


In [39]:
# Membuat Folder Split Dataset
def create_split_dirs():
    for split in ["train", "val", "test"]:
        for cls in CLASSES:
            os.makedirs(os.path.join(FINAL_DATASET, split, cls), exist_ok=True)

create_split_dirs()

In [40]:
# Split Dataset (70 / 15 / 15)
def split_dataset():
    for cls in CLASSES:
        images = os.listdir(os.path.join(RESIZED_DATASET, cls))
        random.shuffle(images)

        total = len(images)
        train_end = int(total * TRAIN_RATIO)
        val_end = train_end + int(total * VAL_RATIO)

        for img in images[:train_end]:
            shutil.copy(
                os.path.join(RESIZED_DATASET, cls, img),
                os.path.join(FINAL_DATASET, "train", cls, img)
            )

        for img in images[train_end:val_end]:
            shutil.copy(
                os.path.join(RESIZED_DATASET, cls, img),
                os.path.join(FINAL_DATASET, "val", cls, img)
            )

        for img in images[val_end:]:
            shutil.copy(
                os.path.join(RESIZED_DATASET, cls, img),
                os.path.join(FINAL_DATASET, "test", cls, img)
            )

split_dataset()

In [41]:
# Augmentasi Offline (Train Only)
TARGET_SAMPLES = {
    "cardboard": 300,
    "glass": 300,
    "metal": 300,
    "paper": 300,
    "plastic": 300,
    "trash": 300
}

def augment_train_data_balanced():
    train_dir = os.path.join(FINAL_DATASET, "train")

    for cls in CLASSES:
        cls_path = os.path.join(train_dir, cls)
        images = [img for img in os.listdir(cls_path) if not img.startswith("aug_")]
        current_count = len(images)
        target_count = TARGET_SAMPLES[cls]

        print(f"\nClass: {cls}")
        print(f"Current: {current_count}, Target: {target_count}")

        if current_count >= target_count:
            print("→ Tidak perlu augmentasi")
            continue

        aug_needed = target_count - current_count
        aug_per_image = math.ceil(aug_needed / current_count)

        aug_index = 0

        for img_name in tqdm(images, desc=f"Augmenting {cls}"):
            img_path = os.path.join(cls_path, img_name)
            img = Image.open(img_path).convert("RGB")
            base, ext = os.path.splitext(img_name)

            for i in range(aug_per_image):
                if aug_index >= aug_needed:
                    break

                aug_img = img.copy()

                # Pola augmentasi bergilir (aman untuk TrashNet)
                mode = i % 4

                # 1. Horizontal Flip
                if mode == 0:
                    aug_img = aug_img.transpose(Image.FLIP_LEFT_RIGHT)

                # 2. Vertical Flip
                elif mode == 1:
                    aug_img = aug_img.transpose(Image.FLIP_TOP_BOTTOM)

                # 3. Rotasi kecil ±5°
                elif mode == 2:
                    aug_img = aug_img.rotate(random.choice([-5, 5]))

                # 4. Rotasi kecil + horizontal flip
                elif mode == 3:
                    aug_img = aug_img.rotate(random.choice([-5, 5]))
                    aug_img = aug_img.transpose(Image.FLIP_LEFT_RIGHT)

                aug_filename = f"aug_{base}_{aug_index}{ext}"
                aug_img.save(os.path.join(cls_path, aug_filename))

                aug_index += 1

        print(f"Augmentasi selesai untuk {cls}: total {current_count + aug_index}")

# Jalankan augmentasi
augment_train_data_balanced()


Class: cardboard
Current: 282, Target: 300


Augmenting cardboard: 100%|█████████████████████████████████████████████████████████| 282/282 [00:01<00:00, 211.53it/s]


Augmentasi selesai untuk cardboard: total 300

Class: glass
Current: 350, Target: 300
→ Tidak perlu augmentasi

Class: metal
Current: 286, Target: 300


Augmenting metal: 100%|█████████████████████████████████████████████████████████████| 286/286 [00:01<00:00, 270.73it/s]


Augmentasi selesai untuk metal: total 300

Class: paper
Current: 415, Target: 300
→ Tidak perlu augmentasi

Class: plastic
Current: 336, Target: 300
→ Tidak perlu augmentasi

Class: trash
Current: 95, Target: 300


Augmenting trash: 100%|███████████████████████████████████████████████████████████████| 95/95 [00:00<00:00, 164.07it/s]

Augmentasi selesai untuk trash: total 300


In [42]:
# Data Generator (RESCALE 1/255)
BATCH_SIZE = 32

train_gen = ImageDataGenerator(
    rescale=1./255
)

val_test_gen = ImageDataGenerator(
    rescale=1./255
)

train_data = train_gen.flow_from_directory(
    os.path.join(FINAL_DATASET, "train"),
    target_size=(224, 224),
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

val_data = val_test_gen.flow_from_directory(
    os.path.join(FINAL_DATASET, "val"),
    target_size=(224, 224),
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

test_data = val_test_gen.flow_from_directory(
    os.path.join(FINAL_DATASET, "test"),
    target_size=(224, 224),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

Found 2001 images belonging to 6 classes.
Found 377 images belonging to 6 classes.
Found 383 images belonging to 6 classes.


In [43]:
# Model MobileNet (Transfer Learning)
base_model = MobileNet(
    weights="imagenet",
    include_top=False,
    input_shape=(224, 224, 3)
)

base_model.trainable = False

In [44]:
# Build & Compile Model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(64, activation="relu"),
    Dense(len(CLASSES), activation="softmax")
])

model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ mobilenet_1.00_224 (Functional)      │ (None, 7, 7, 1024)          │       3,228,864 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_3           │ (None, 1024)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 64)                  │          65,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 6)                   │             390 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,294,854 (12.57 MB)

 Trainable params: 65,990 (257.77 KB)

 Non-trainable params: 3,228,864 (12.32 MB)

In [45]:
# Implementasi Early Stopping
early_stopping = EarlyStopping(
    monitor="val_loss",
    patience=3,
    restore_best_weights=True,
    verbose=1
)

In [46]:
# Training Model + Pengukuran Waktu Komputasi
EPOCHS = 20

start_time = time.time()

history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS,
    callbacks=[early_stopping],
    verbose=1
)

end_time = time.time()

total_training_time = end_time - start_time
epochs_ran = len(history.history["loss"])

print(f"\nTotal Training Time: {total_training_time:.2f} seconds")
print(f"Average Time per Epoch: {total_training_time / epochs_ran:.2f} seconds")
print(f"Training stopped at epoch: {epochs_ran}")

Epoch 1/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 21s 308ms/step - accuracy: 0.3418 - loss: 1.6436 - val_accuracy: 0.5013 - val_loss: 1.3256
Epoch 2/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 14s 226ms/step - accuracy: 0.5792 - loss: 1.1956 - val_accuracy: 0.6313 - val_loss: 1.0654
Epoch 3/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 14s 225ms/step - accuracy: 0.6742 - loss: 0.9584 - val_accuracy: 0.6711 - val_loss: 0.9138
Epoch 4/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 14s 227ms/step - accuracy: 0.7291 - loss: 0.8191 - val_accuracy: 0.7268 - val_loss: 0.8202
Epoch 5/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 14s 226ms/step - accuracy: 0.7491 - loss: 0.7350 - val_accuracy: 0.7480 - val_loss: 0.7572
Epoch 6/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 14s 227ms/step - accuracy: 0.7751 - loss: 0.6672 - val_accuracy: 0.7480 - val_loss: 0.7070
Epoch 7/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 14s 226ms/step - accuracy: 0.7961 - loss: 0.6117 - val_accuracy: 0.7666 - val_loss: 0.6719
Epoch 8/20
63/63 ━━━━━━━━━━━━━━━━━━━━ 14s 228ms/step - accuracy: 0.8126 - loss: 0.5719 - val_accu

In [47]:
# Waktu Inferensi Keseluruhan Test Set
start_time = time.time()

predictions = model.predict(test_data)

end_time = time.time()

inference_time_total = end_time - start_time
num_samples = test_data.samples

print(f"Total Inference Time (Test Set): {inference_time_total:.4f} seconds")
print(f"Average Inference Time per Image: {inference_time_total / num_samples:.6f} seconds")

12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 321ms/step
Total Inference Time (Test Set): 4.3847 seconds
Average Inference Time per Image: 0.011448 seconds


In [48]:
# Evaluasi Model (Test Set – Tetap Sama)
test_loss, test_acc = model.evaluate(test_data)
print(f"Test Accuracy: {test_acc:.4f}")

12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 190ms/step - accuracy: 0.7963 - loss: 0.5408
Test Accuracy: 0.7963


In [49]:
# Ambil 1 batch
x_batch, _ = next(test_data)

# Warm-up (penting untuk CNN)
_ = model.predict(x_batch[:1])

start_time = time.time()
_ = model.predict(x_batch[:1])
end_time = time.time()

print(f"Inference Time (Single Image): {(end_time - start_time):.6f} seconds")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 349ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Inference Time (Single Image): 0.057216 seconds


In [50]:
# CONFUSION MATRIX & REPORT
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

y_true = test_data.classes
y_pred = np.argmax(model.predict(test_data), axis=1)

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=CLASSES))

print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred))

12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 188ms/step

Classification Report:
              precision    recall  f1-score   support

   cardboard       0.89      0.79      0.83        61
       glass       0.71      0.75      0.73        76
       metal       0.84      0.84      0.84        62
       paper       0.85      0.90      0.88        90
     plastic       0.77      0.75      0.76        72
       trash       0.59      0.59      0.59        22

    accuracy                           0.80       383
   macro avg       0.78      0.77      0.77       383
weighted avg       0.80      0.80      0.80       383


Confusion Matrix:
[[48  1  1  8  0  3]
 [ 0 57  6  2 11  0]
 [ 2  4 52  3  1  0]
 [ 4  0  0 81  3  2]
 [ 0 12  2  0 54  4]
 [ 0  6  1  1  1 13]]


In [51]:
MODEL_PATH = "mobilenetv1_trashnet.keras"
model.save(MODEL_PATH)

print(f"Model berhasil disimpan di: {MODEL_PATH}")

Model berhasil disimpan di: mobilenetv1_trashnet.keras
